In [3]:
import os
import sys

current_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(current_dir, '..', 'tokenizer'))
sys.path.append(module_path)

from tokenizerv2 import TokenizerV2
tokenizer = TokenizerV2()
tokenizer.load_object('../tokenizer.pkl')

In [12]:
from datasets import load_dataset
from tqdm import tqdm

ds = load_dataset("stanfordnlp/imdb")

train = ds['train']
test = ds['test']

# Batch processing for train data with a progress bar
X_train = [tokenizer.encode(text) for text in tqdm(train['text'], desc="Encoding train data")]
y_train = train['label']





Encoding train data: 100%|██████████| 25000/25000 [00:15<00:00, 1650.40it/s]


In [19]:
print(X_train[:5])

[[642, 186, 209, 865, 642, 248, 962, 315, 515, 24, 375, 211, 89, 24, 346, 35, 120, 68, 493, 493, 89, 432, 315, 265, 245, 315, 692, 806, 964, 181, 79, 943, 551, 558, 79, 29, 315, 1068, 315, 105, 315, 189, 1002, 845, 801, 1081, 315, 338, 506, 315, 444, 70, 1004, 753, 806, 865, 328, 315, 17, 482, 315, 328, 315, 861, 370, 280, 70, 46, 315, 596, 761, 559, 865, 104, 169, 378, 948, 315, 135, 642, 446, 204, 315, 584, 478, 338, 506, 315, 506, 315, 280, 70, 46, 315, 328, 315, 861, 370, 559, 101, 987, 315, 563, 24, 315, 135, 346, 135, 953, 46, 289, 370, 347, 328, 315, 779, 315, 845, 467, 987, 315, 3, 315, 986, 794, 315, 338, 814, 315, 189, 753, 100, 656, 105, 22, 693, 79, 943, 854, 315, 544, 210, 239, 29, 315, 280, 364, 455, 189, 1002, 733, 806, 1045, 542, 189, 1002, 845, 801, 733, 784, 542, 642, 1090, 364, 365, 1010, 3, 315, 447, 315, 338, 814, 315, 693, 315, 245, 981, 135, 704, 624, 315, 33, 381, 704, 624, 315, 33, 381, 465, 79, 27, 814, 315, 740, 1002, 209, 1045, 260, 544, 972, 819, 518, 605, 

In [34]:
#find max in X_train
max_id = [max(x) for x in X_train]
max_id = max(max_id)
print(max_id)
print(tokenizer.vocab_size)
print(len(X_train))
print(X_train[:5])
min_id = [min(x) for x in X_train]
print(min(min_id))

1109
1111
25000
[[642, 186, 209, 865, 642, 248, 962, 315, 515, 24, 375, 211, 89, 24, 346, 35, 120, 68, 493, 493, 89, 432, 315, 265, 245, 315, 692, 806, 964, 181, 79, 943, 551, 558, 79, 29, 315, 1068, 315, 105, 315, 189, 1002, 845, 801, 1081, 315, 338, 506, 315, 444, 70, 1004, 753, 806, 865, 328, 315, 17, 482, 315, 328, 315, 861, 370, 280, 70, 46, 315, 596, 761, 559, 865, 104, 169, 378, 948, 315, 135, 642, 446, 204, 315, 584, 478, 338, 506, 315, 506, 315, 280, 70, 46, 315, 328, 315, 861, 370, 559, 101, 987, 315, 563, 24, 315, 135, 346, 135, 953, 46, 289, 370, 347, 328, 315, 779, 315, 845, 467, 987, 315, 3, 315, 986, 794, 315, 338, 814, 315, 189, 753, 100, 656, 105, 22, 693, 79, 943, 854, 315, 544, 210, 239, 29, 315, 280, 364, 455, 189, 1002, 733, 806, 1045, 542, 189, 1002, 845, 801, 733, 784, 542, 642, 1090, 364, 365, 1010, 3, 315, 447, 315, 338, 814, 315, 693, 315, 245, 981, 135, 704, 624, 315, 33, 381, 704, 624, 315, 33, 381, 465, 79, 27, 814, 315, 740, 1002, 209, 1045, 260, 544, 972,

In [46]:
import numpy as np
def td_idf(X,vocab_size):
    N = len(X)
    if N == 0 or vocab_size == 0:
        return np.zeros((N, vocab_size), dtype=np.float32)
    tf_raw = np.zeros((N, vocab_size), dtype=np.float32)
    df = np.zeros((vocab_size,), dtype=np.float32)
    for i, doc in enumerate(X):
        if not doc:
            continue
        unique_words = np.unique(doc, return_counts=True)
        for word, count in zip(*unique_words):
            tf_raw[i, word] = count
            df[word] += 1
    idf = np.log(N / (1 + df))
    print(f"Statistiques DF: Min={np.min(df)}, Max={np.max(df)}, Moyenne={np.mean(df)}")
    print(f"Nombre de DF > 0: {np.count_nonzero(df)}")
    # Regardez les valeurs IDF aussi
    print(f"Statistiques IDF: Min={np.min(idf)}, Max={np.max(idf)}, Moyenne={np.mean(idf)}")
    tfidf_vectors = tf_raw * idf
    norm = np.linalg.norm(tfidf_vectors, axis=1, keepdims=True)
    tfidf_vectors_normalized = np.divide(tfidf_vectors, norm, out=np.zeros_like(tfidf_vectors), where=norm!=0)
    return tfidf_vectors_normalized


In [54]:
tf_idf_train = td_idf(X_train,tokenizer.vocab_size)

Statistiques DF: Min=0.0, Max=25000.0, Moyenne=4484.67041015625
Nombre de DF > 0: 666
Statistiques IDF: Min=-3.999551699962467e-05, Max=10.126630783081055, Moyenne=5.144969940185547


In [55]:
print(len(tf_idf_train))
print(tokenizer.vocab_size)

25000
1111


In [56]:
# Save the final vocabulary to a file as a numpy array
np.save('tf_idf_train.npy', tf_idf_train)

In [57]:
def transform_tfidf(X_ids, vocab_size, idf_train):
    N_test = len(X_ids)
    if N_test == 0 or vocab_size == 0:
        return np.zeros((N_test, vocab_size), dtype=np.float32)

    tf_raw_test = np.zeros((N_test, vocab_size), dtype=np.float32)

    for i, doc in enumerate(X_ids):
        if not doc: continue
        unique_ids, counts = np.unique(doc, return_counts=True)
        for word_id, count in zip(unique_ids, counts):
            if 0 <= word_id < vocab_size:
                tf_raw_test[i, word_id] = count
        if (i + 1) % 5000 == 0: print(f"  Doc test {i+1}/{N_test} traité.")

    # Utiliser les scores IDF appris sur le train
    tfidf_vectors_test = tf_raw_test * idf_train # Broadcasting de idf_train

    norm_test = np.linalg.norm(tfidf_vectors_test, axis=1, keepdims=True)
    tfidf_test_normalized = np.divide(tfidf_vectors_test, norm_test, out=np.zeros_like(tfidf_vectors_test), where=norm_test!=0)

    return tfidf_test_normalized

In [58]:
X_test = [tokenizer.encode(text) for text in tqdm(test['text'], desc="Encoding test data")]
y_test = test['label']
test_tf_idf = transform_tfidf(X_test,tokenizer.vocab_size,tf_idf_train)

Encoding test data: 100%|██████████| 25000/25000 [00:14<00:00, 1722.25it/s]


  Doc test 5000/25000 traité.
  Doc test 10000/25000 traité.
  Doc test 15000/25000 traité.
  Doc test 20000/25000 traité.
  Doc test 25000/25000 traité.


In [59]:
np.save('tf_idf_test.npy', test_tf_idf)